Парсинг группы ВКонтакте "Большой МИР маленьких ДЕТЕЙ"
(mir.detey)

In [1]:
!pip install vk_api
!pip install tqdm
!pip install pandas
!pip install nltk
!pip install pymystem3
!pip install wordcloud
!pip install matplotlib
!pip install scikit-learn
!pip install TextBlob

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


Подключаем api вконтакте для парсинга группы

In [2]:
import vk_api
import math
import datetime
import time
import pandas as pd
from tqdm import tqdm

session = vk_api.VkApi(token='<СЕКРЕТНЫЙ ТОКЕН КОТОРЫЙ НЕЛЬЗЯ ПУБЛИКОВАТЬ>')

vk = session.get_api()

Подготавливаю модели для оперирования в коде. Из-за того что данные которые возвращает методы апи имеют избыточность, то нам необходимо подготовить свои модели, для того чтобы сохранить только конкретные данные.


Группа

id - идентификатор группы int

name - имя группы на русском языке string

screen_name - имя группы на латинице

In [2]:
class vk_group:
    def __init__(self, id: int, name: str, screen_name: str):
        self.id = id
        self.name = name
        self.screen_name = screen_name


Посты группы


id - идентификатор поста int

group_id - идентификатор группы int

date - дата публикации datetime

text - текст поста в группе string

comments_count - количество комментариев int

In [3]:
class vk_group_post:
    def __init__(self, id: int, group_id: int, date: datetime, text: str, comments_count: int):
        self.id = id
        self.group_id = group_id
        self.date = date
        self.text = text
        self.comments_count = comments_count

Комментарии под постом группы


id - идентификатор комментария int

post_id - идентификатор поста группы int

parent_id - идентификатор родительского комментария int

date - дата публикации datetime

user_id - идентификатор пользователя int

text - текст комментария

In [4]:
class vk_group_post_comment:
    def __init__(self, id: int, post_id: int, parent_id: int, date: datetime, user_id: int, sub_comments_count: int, text: str):
        self.id = int(id)
        self.post_id = post_id
        self.parent_id = parent_id
        self.date = date
        self.user_id = user_id
        self.sub_comments_count = sub_comments_count
        self.text = text

Получить информацию о группе по имени группы

https://dev.vk.com/ru/method/groups.search

In [5]:
def get_group_info_by_name(group_name):
    vk_group_info = None;
    try:
        group_info_from_api = vk.groups.search(q=group_name, type='group', count=1, v='5.199')['items'][0]
        vk_group_info = vk_group(group_info_from_api["id"], group_info_from_api["name"], group_info_from_api["screen_name"])
    except vk_api.exceptions.ApiError as error:
        print(f"An error occurred: {error}")

    return vk_group_info

print(vars(get_group_info_by_name('mir.detey')))

{'id': 83421847, 'name': 'Большой МИР маленьких ДЕТЕЙ', 'screen_name': 'mir.detey'}


Получить информацию о группе по идентификатору

https://dev.vk.com/ru/method/groups.getById

In [6]:
def get_group_info_by_id(group_id):
    vk_group_info = None;
    try:
        group_info_from_api = vk.groups.getById(group_id=group_id, v='5.199')['groups'][0]
        vk_group_info = vk_group(group_info_from_api["id"], group_info_from_api["name"], group_info_from_api["screen_name"])
    except vk_api.exceptions.ApiError as error:
        print(f"An error occurred: {error}")

    return vk_group_info

print(vars(get_group_info_by_id(83421847)))

{'id': 83421847, 'name': 'Большой МИР маленьких ДЕТЕЙ', 'screen_name': 'mir.detey'}


Получить посты в группе по её идентификатору

https://dev.vk.com/ru/method/wall.get

In [7]:
def get_posts_by_group_id(group_id, offset = 0):
    posts = []
    posts_count = 0
    try:
        response = vk.wall.get(owner_id=f'-{group_id}', count=100, offset=offset, v='5.199')
        posts_count = response['count']
        posts_from_api = response['items']
        posts = [vk_group_post(post['id'], group_id, datetime.datetime.fromtimestamp(post['date']), post['text'], post['comments']['count']) for post in posts_from_api]
    except vk_api.exceptions.ApiError as error:
        print(f"An error occurred: {error}")

    return posts, posts_count

Получить комментарии по посту

https://dev.vk.com/ru/method/wall.getComments


In [8]:
def get_comments_by_post_id(group_id, post_id, offset = 0):
    comments = []
    try:
        response = vk.wall.getComments(owner_id=f'-{group_id}', post_id=post_id, count=100, offset=offset, v='5.199')
        comments_from_api = response['items']
        comments = [vk_group_post_comment(comment['id'], post_id, 0, datetime.datetime.fromtimestamp(comment['date']), comment['from_id'], comment['thread']['count'], comment['text']) for comment in comments_from_api]
    except vk_api.exceptions.ApiError as error:
        print(f"An error occurred: {error}")

    return comments

def get_sub_comments_by_comment_id(group_id, post_id, comment_id, offset = 0):
    sub_comments = []
    try:
        response = vk.wall.getComments(owner_id=f'-{group_id}', post_id=post_id, comment_id=comment_id,  count=100, offset=offset, v='5.199')
        sub_comments_from_api = response['items']
        sub_comments = [vk_group_post_comment(sub_comment['id'], post_id, comment_id, datetime.datetime.fromtimestamp(sub_comment['date']), sub_comment['from_id'], 0, sub_comment['text']) for sub_comment in sub_comments_from_api]
    except vk_api.exceptions.ApiError as error:
        print(f"An error occurred: {error}")

    return sub_comments

Метод сохранения списка в csv

In [ ]:
def save_list_to_csv(file_name, objects):
    data = [obj.__dict__ for obj in objects]
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False, sep=';', encoding='utf-8', header=True)

Получаем все посты и сохраняем в переменную posts и сохраним в файл posts.csv

In [ ]:
posts, posts_count = get_posts_by_group_id(83421847)

if posts_count > 0:
    current_offset = 100
    total_post_pages = math.ceil(posts_count / 100)
    for page in tqdm(range(1, total_post_pages)):
        current_posts, current_posts_count = get_posts_by_group_id(83421847, current_offset)
        posts.extend(current_posts)
        current_offset += 100
        time.sleep(1)

save_list_to_csv('posts.csv', posts)

Получаем комментарии и подкомментарии к каждому посту и сохраняем в переменную comments и сохраним в файл comments.csv

In [ ]:
comments = []
for post in tqdm(posts):
    current_comment_offset = 0
    total_comments_pages = math.ceil(post.comments_count / 100)
    for comment_page in range(total_comments_pages):
        current_comments = get_comments_by_post_id(83421847, post.id, current_comment_offset)
        comments.extend(current_comments)
        for comment in current_comments:
            current_sub_comment_offset = 0
            total_sub_comments_pages = math.ceil(comment.sub_comments_count / 100)
            for sub_comments_page in range(total_sub_comments_pages):
                current_sub_comments = get_sub_comments_by_comment_id(83421847, post.id, comment.id, current_sub_comment_offset)
                comments.extend(current_sub_comments)
                current_sub_comment_offset += 100
                time.sleep(1)
        current_comment_offset += 100
        time.sleep(1)

save_list_to_csv('comments.csv', comments)